In [6]:
import pandas as pd
from matplotlib import pyplot as plt


panel_data_df = pd.read_pickle('data/panel_data.pkl')
print(panel_data_df)



          index Maturity 1 Maturity 2 Maturity 3 Maturity 4 Maturity 5
0    1988-01-04      187.5      193.5     198.25      200.0     202.75
1    1988-01-05      187.5     193.25     197.25     198.25     200.75
2    1988-01-06      191.0     195.75     199.75     201.75      204.5
3    1988-01-07     192.75     198.75     202.75      204.5      208.5
4    1988-01-08      192.5      198.0      202.0      203.5     207.25
...         ...        ...        ...        ...        ...        ...
9051 2023-12-22      473.0     485.75      495.5     498.25      504.5
9052 2023-12-26     480.25     492.25      501.5      503.5      509.5
9053 2023-12-27      476.5      488.5      498.0     500.25      506.5
9054 2023-12-28     474.25      486.5      496.0      498.5     504.75
9055 2023-12-29     471.25      484.0      494.0      497.0      503.5

[9076 rows x 6 columns]
